In [ ]:
from functions import *
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KernelDensity
from tqdm import tqdm
import math
import cmath
from scipy.integrate import simpson, romb
from scipy.stats import iqr, norm, gaussian_kde
from scipy.special import erfc, wofz
import pickle
import os

In [ ]:
theta_list, x_list = np.loadtxt("DataPhase28.dat", unpack=True)
T = len(theta_list)
alpha = 0.3
r2 = 0.925

L_limit = 4
n_precision = 10000
X_precision = np.linspace(-L_limit, L_limit, n_precision)
N_theta = 20

In [ ]:
grouped_theta_list, grouped_data_dict = data_wrapper(theta_list, x_list, N_theta)

### with $\mu$ and fixed $\nu$

In [ ]:
mu_max = 8
nu = 1
kappa = 0.9
N_mu = 100
mu_list = np.linspace(-mu_max, mu_max, N_mu)


phi = np.arctan(nu/mu_list)
theta_list_from_mu = np.where(phi<=0, phi, phi + np.pi)


eps = 1e-12
cos_th = np.cos(theta_list_from_mu)
sin_th = np.sin(theta_list_from_mu)
r_list = np.where(np.abs(cos_th) > eps, mu_list / cos_th, nu / sin_th)

# store one solution per mu
solutions_mu_nu = list(zip(mu_list, r_list, theta_list_from_mu))


true_phi_list = []
estimated_gaussian_list = []
corrected_gaussian_list = []
theta_bin_list = []


for mu, radius, theta in tqdm(solutions_mu_nu):
    
    true_phi_list.append(phi_1(theta, alpha, r2, radius, t=1))
    theta_bin = int(np.floor(np.mod(theta, np.pi) / (np.pi / N_theta)))
    xb = np.array(grouped_data_dict[theta_bin])
    W_Gaussian, bw_Gaussian = pdf_kernel_optimal_bandwidth(X_precision, xb)
    phi_t1_vals_gaussian, optimal_bd = compute_gaussian_phi_at_t_optimal(mu / radius, nu / radius, X_precision, xb, W_Gaussian, t=radius)
    estimated_gaussian_list.append(phi_t1_vals_gaussian)
    theta_bin_list.append(theta_bin)

    
    phi_t1_vals_gaussian, optimal_bd = compute_gaussian_phi_at_t_optimal(mu / radius, nu / radius, X_precision, xb, W_Gaussian, t = radius / kappa)
    phi_t1_vals_gaussian *= 1 / np.exp( - ((1 - kappa) / kappa)**2 / 2 )
    corrected_gaussian_list.append(phi_t1_vals_gaussian)

In [ ]:
plt.plot(mu_list, np.array(true_phi_list).real, label='True $Re[\phi(t=1)]$', linewidth = 2)
plt.plot(mu_list, np.array(true_phi_list).imag, label='True $Im[\phi(t=1)]$', linewidth = 2)

plt.plot(mu_list, np.array(estimated_gaussian_list).real, '--', label='Estimated $Re[\phi(t=1)]$', linewidth = 2)
plt.plot(mu_list, np.array(estimated_gaussian_list).imag, '--', label='Estimated $Im[\phi(t=1)]$', linewidth = 2)

plt.plot(mu_list, np.array(corrected_gaussian_list).real, '-x', label='Corrected $Re[\phi(t=1)]$', linewidth = 2)
plt.plot(mu_list, np.array(corrected_gaussian_list).imag, '-x', label='Corrected $Im[\phi(t=1)]$', linewidth = 2)

plt.legend()
plt.xlabel(r"$\mu$")
plt.ylabel(r"$\phi(1, \mu, \nu)$")
plt.title(rf"$\phi(1,\mu,\nu)$, $\nu$ = {nu}")
plt.ylim(-1, 1)